<a href="https://colab.research.google.com/github/amiholz/NN/blob/master/Ex1_q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)


test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 5, activation='relu')
        self.conv2 = Conv2D(64, 5, activation='relu')
        self.max_pooling1 = MaxPooling2D()
        self.max_pooling2 = MaxPooling2D()
        self.flatten = Flatten()
        self.d1 = Dense(1024, activation='relu')  # kernel_initializer='random_normal'
        self.d2 = Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.max_pooling1(x)
        x = self.conv2(x)
        x = self.max_pooling2(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)



11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Create an instance of the model
model = MyModel()


loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)



In [0]:
EPOCHS = 5
train_iter_num = 0
TRAIN_ITERS_FOR_PRINT = 500

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
num_of_iters = []

inside_epoch_template = 'Inside Epoch {}, Total Iters: {}, Train Loss: {}, Train Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
end_epoch_template = 'Ending Epoch {}, Total Iters: {},  Train Loss: {}, Train Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
        train_iter_num += 1
        if train_iter_num % TRAIN_ITERS_FOR_PRINT == 0:
              
              for (test_images, test_labels) in test_ds:
                    test_step(test_images, test_labels)
              
              # train_losses.append(train_loss.result())
              # train_accuracies.append(train_accuracy.result()*100)
              # test_losses.append(test_loss.result())
              # test_accuracies.append(test_accuracy.result()*100))
              # num_of_iters.append(train_iter_num)
              
              #print(inside_epoch_template.format(epoch+1,train_iter_num,
                              # train_loss.result(),
                              # train_accuracy.result()*100,
                              # test_loss.result(),
                              # test_accuracy.result()*100))
    
    for (test_images, test_labels) in test_ds:
        test_step(test_images, test_labels)
           
    print(end_epoch_template.format(epoch+1,
                          train_iter_num,
                              train_loss.result(),
                              train_accuracy.result()*100,
                              test_loss.result(),
                              test_accuracy.result()*100))
    # test_accuracies.append()
    print(" -- ", flush=True)
    
    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Ending Epoch 1, Total Iters: 1875,  Train Loss: 0.06661814451217651, Train Accuracy: 97.98916625976562, Test Loss: 0.04841623827815056, Test Accuracy: 98.49500274658203
 -- 
Ending Epoch 2, Total Iters: 3750,  Train Loss: 0.02511233650147915, Train Accuracy: 99.25666809082031, Test Loss: 0.032346341758966446, Test Accuracy: 98.98999786376953
 -- 
Ending Epoch 3, Total Iters: 5625,  Train Loss: 0.018462292850017548, Train Accuracy: 99.38500213623047, Test Loss: 0.03245525807142258, Test Accuracy: 99.04399871826172
 -- 
Ending Epoch 4, Total Iters: 7500,  Train Loss: 0.015113821253180504, Train Accuracy: 99.54499816894531, Test Loss: 0.032999247312545776, Test Accuracy: 99.10800170898438
 -- 
Ending Epoch 5, Total Iters: 9375,  Train Loss: 0.013624678365886211, Train Accuracy: 99.54666137695312, Test Loss: 0.038215503096580505, Test Accuracy: 99.04500579833984
 -- 


In [0]:
from matplotlib import pyplot as plt